In [ ]:
# %load_ext lab_black
%load_ext autoreload
%autoreload 1

### Imports

Use `aimport` for `schedview` imports for ease of debugging.

In [ ]:
import warnings
import math
import os
import sys
import logging
from pathlib import Path
import panel as pn
import numpy as np
import pandas as pd
import param
import bokeh
from copy import deepcopy
import datetime
from pytz import timezone
import lzma
import pickle
import yaml
import json
import socket
import time
from contextlib import redirect_stdout
from pathlib import Path
from collections import OrderedDict
from tempfile import TemporaryDirectory, NamedTemporaryFile
import hashlib
import shutil

import sys
from conda.exceptions import EnvironmentLocationNotFound
from conda.gateways.disk.test import is_conda_environment
from conda.cli.main_list import print_explicit, list_packages

In [ ]:
from astropy.time import Time, TimeDelta
from zoneinfo import ZoneInfo
import matplotlib as mpl
import matplotlib.pyplot as plt
import hvplot.pandas

In [ ]:
import lsst.resources

In [ ]:
import rubin_scheduler
from rubin_scheduler.scheduler.example import example_scheduler
from rubin_scheduler.scheduler import sim_runner
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from rubin_scheduler.scheduler.utils import SchemaConverter

In [ ]:
import schedview
import schedview.app.prenight
import schedview.compute.scheduler
import schedview.collect.opsim
from schedview.plot.visitmap import BAND_COLORS

### Further preparation of the notebook

In [ ]:
pn.extension("terminal")

### Filter warnings

Several dependencies throw prodigious instances of (benign) warnings.
Suppress them to avoid poluting the executed notebook.

In [ ]:
warnings.filterwarnings(
    "ignore",
    module="astropy.time",
    message="Numerical value without unit or explicit format passed to TimeDelta, assuming days",
)
warnings.filterwarnings(
    "ignore",
    module="pandas",
    message="In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.",
)
warnings.filterwarnings(
    "ignore",
    module="healpy",
    message="divide by zero encountered in divide",
)
warnings.filterwarnings(
    "ignore",
    module="healpy",
    message="invalid value encountered in multiply",
)
warnings.filterwarnings(
    "ignore",
    module="holoviews",
    message="Discarding nonzero nanoseconds in conversion.",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim",
    message="invalid value encountered in arcsin",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim",
    message="All-NaN slice encountered",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim.scheduler.utils",
    message="invalid value encountered in cast",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_scheduler.scheduler.utils",
    message="invalid value encountered in cast",
)

## Configuration and initial configuration

Setting `keep_rewards` to `True` results in a dashboard that includes plots of rewards.

In [ ]:
keep_rewards = True

Set the start date, scheduler, and observatory for the night:

In [ ]:
observatory = ModelObservatory()

Set `evening_mjd` to the integer calendar MJD of the local calendar day on which sunset falls on the night of interest.

In [ ]:
evening_iso8601 = "2025-01-01"

night_date = datetime.date.fromisoformat(evening_iso8601)
evening_mjd = Time(evening_iso8601).mjd
night_date, evening_mjd

If we just use this day as the start and make the simulation duration 1 day, the begin and end of the simulation will probably begin in the middle on one night and end in the middle of the next.
Instead, find the sunset and sunrise of the night we want using the almanac, and use these to determine our start time and duration.

In [ ]:
# If the date represents the local calendar date at sunset, we need to shift by the longitude in units of days
this_night = (
    np.floor(observatory.almanac.sunsets["sunset"] + observatory.site.longitude / 360)
    == evening_mjd
)

mjd_start = observatory.almanac.sunsets[this_night]["sun_n12_setting"][0]
mjd_end = observatory.almanac.sunsets[this_night]["sunrise"][0]

night_duration = mjd_end - mjd_start
time_start = Time(mjd_start, format="mjd")
time_start.iso, night_duration

In [ ]:
observatory = ModelObservatory(mjd_start=mjd_start)

In [ ]:
scheduler = example_scheduler(mjd_start=mjd_start)

Record the date of local day in the evening. 

## Prepare the output dir

In [ ]:
data_dir = TemporaryDirectory()
data_path = Path(data_dir.name)



## Save the starting scheduler

In [ ]:
scheduler_fname = data_path.joinpath('scheduler.pickle.xz')

with lzma.open(scheduler_fname, "wb", format=lzma.FORMAT_XZ) as pio:
    pickle.dump(scheduler, pio)

## Run a simulation and create the app instance

For this example, simulate starting the default first day of observing:

In [ ]:
exec_start_time = Time.now()
if not keep_rewards:
    observatory, scheduler, observations = sim_runner(
        observatory, scheduler, mjd_start=mjd_start, survey_length=night_duration
    )
else:
    scheduler.keep_rewards = True
    observatory, scheduler, observations, reward_df, obs_rewards = sim_runner(
        observatory,
        scheduler,
        mjd_start=mjd_start,
        survey_length=night_duration,
        record_rewards=True,
    )

## Save the notebook

In [ ]:
notebook_fname = data_path.joinpath('notebook.ipynb').as_posix()

In [ ]:
%notebook $notebook_fname

## Save the conda environment

In [ ]:
conda_prefix = Path(sys.executable).parent.parent.as_posix()
if not is_conda_environment(conda_prefix):
    raise EnvironmentLocationNotFound(conda_prefix)

environment_fname = data_path.joinpath('environment.txt').as_posix()

In [ ]:
environment_fname

In [ ]:
!conda list --export -p $conda_prefix > $environment_fname

## Archive the results

In [ ]:
archive_files = {'scheduler': scheduler_fname, 'notebook': notebook_fname, 'conda_environment': environment_fname}

In [ ]:
def make_sim_archive_dir(observations, reward_df=None, obs_rewards=None, in_files = {}, data_path=None, destination_uri=None):
    if data_path is None:
        data_dir = TemporaryDirectory()
        data_path = Path(data_dir.name)
    else:
        data_dir = None

        if not isinstance(data_path, Path):
            data_path = Path(data_path)

    files = {}
    
    # Save the observations
    files['observations'] = {'name': 'opsim.db'}
    opsim_output_fname = data_path.joinpath(files['observations']['name'])
    SchemaConverter().obs2opsim(observations, filename=opsim_output_fname)

    # Save the rewards
    if reward_df is not None and obs_rewards is not None:
        files['rewards'] = {'name': 'rewards.h5'}
        rewards_fname = data_path.joinpath(files['rewards']['name'])
        if reward_df is not None:
            reward_df.to_hdf(rewards_fname, "reward_df")
        if obs_rewards is not None:
            obs_rewards.to_hdf(rewards_fname, "obs_rewards")

    # Save basic statistics
    files['statistics'] = {'name': 'obs_stats.txt'}
    stats_fname = data_path.joinpath(files['statistics']['name'])
    with open(stats_fname, 'w') as stats_io:
        print(SchemaConverter().obs2opsim(observations).describe().T.to_csv(sep='\t'), file=stats_io)

    # Add supplied files
    for file_type, fname in in_files.items():
        files[file_type] = {'name': Path(fname).name}
        try:
            shutil.copyfile(fname, data_path.joinpath(files[file_type]['name']))
        except shutil.SameFileError:
            pass
    
    # Add file hashes
    for file_type in files:
        fname = data_path.joinpath(files[file_type]['name'])
        with open(fname, 'rb') as file_io:
            content = file_io.read()

        files[file_type]['md5'] = hashlib.md5(content).hexdigest()
    
    # Metadata
    def evening_local_date(mjd, longitude=observatory.site.longitude):
        evening_local_mjd = np.floor(mjd + longitude/360 - 0.5).astype(int)
        evening_local_iso = Time(evening_local_mjd, format='mjd').iso[:10]
        return evening_local_iso

    opsim_metadata = {}
    opsim_metadata['scheduler_version'] = rubin_scheduler.__version__
    opsim_metadata['host'] = socket.getfqdn()
    opsim_metadata['username'] = os.environ['USER']
    opsim_metadata['simulated_dates'] = {}
    opsim_metadata['simulated_dates']['start'] = evening_local_date(mjd_start)
    opsim_metadata['simulated_dates']['end'] = evening_local_date(mjd_start + night_duration)
    opsim_metadata['files'] = files
    
    sim_metadata_fname = data_path.joinpath('sim_metadata.yaml')
    with open(sim_metadata_fname, 'w') as sim_metadata_io:
        print(yaml.dump(opsim_metadata, indent=4), file=sim_metadata_io)

    files['metadata'] = {'name': sim_metadata_fname}

    return data_dir


In [ ]:
archive_dir = make_sim_archive_dir(observations, reward_df, obs_rewards, in_files=archive_files)
archive_dir.name

In [ ]:
!ls -alth $archive_dir.name

In [ ]:
os.environ['S3_ENDPOINT_URL'] = 'https://s3dfrgw.slac.stanford.edu/'
os.environ['AWS_PROFILE'] = 'prenight'

In [ ]:
def transfer_archive_dir(archive_dir, archive_base_uri='s3://rubin-scheduler-prenight/opsim/'):
    metadata_fname = Path(archive_dir).joinpath('sim_metadata.yaml')
    with open(metadata_fname, 'r') as metadata_io:
        sim_metadata = yaml.safe_load(metadata_io)

    insert_date = datetime.datetime.utcnow().date().isoformat()
    insert_date_rpath = lsst.resources.ResourcePath(archive_base_uri).join(insert_date, forceDirectory=True)
    if not insert_date_rpath.exists():
        insert_date_rpath.mkdir()

    # Number the sims in the insert date dir
    found_ids = []
    for base_dir, found_dirs, found_files in insert_date_rpath.walk():
        if base_dir == insert_date_rpath:
            for found_dir in found_dirs:
                try:
                    found_ids.append(int(found_dir[:-1]))
                except ValueError:
                    pass
    
    new_id = max(found_ids) + 1 if len(found_ids)>0 else 1
    resource_rpath = insert_date_rpath.join(f"{new_id}", forceDirectory=True)
    resource_rpath.mkdir()

    sim_metadata['files']['metadata'] = {'name': 'sim_metadata.yaml'}
    
    for file_type, file_info in sim_metadata['files'].items():
        source_fname = Path(archive_dir).joinpath(file_info['name'])
        with open(source_fname, 'rb') as source_io:
            content = source_io.read()

        destination_rpath = resource_rpath.join(file_info['name'])
        destination_rpath.write(content)

        print(f"Copied {source_fname} to {destination_rpath}")

    return resource_rpath

In [ ]:
transfer_archive_dir(archive_dir.name)

In [ ]:
for base_dir, found_dirs, found_files in lsst.resources.ResourcePath('s3://rubin-scheduler-prenight/opsim/').walk():
    for found_file in found_files:
        print(base_dir, found_file)

In [ ]:
print(lsst.resources.ResourcePath('s3://rubin-scheduler-prenight/opsim/2023-12-15/1/sim_metadata.yaml').read().decode('utf-8'))